In [ ]:
import gdown
import pandas as pd

url = 'https://drive.google.com/uc?id=1-HPPo7XYSL-lQ6D9lfSpDMh76PsBDv71'
output = 'prev.xlsx'

gdown.download(url, output, quiet=False)

df = pd.read_excel(output, engine='openpyxl')


Downloading...
From: https://drive.google.com/uc?id=1-HPPo7XYSL-lQ6D9lfSpDMh76PsBDv71
To: /content/prev.xlsx
100%|██████████| 22.8k/22.8k [00:00<00:00, 20.5MB/s]


FileNotFoundError: [Errno 2] No such file or directory: 'pred.txt'

In [ ]:
!pip install rouge-score
!pip install bert-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=363bf1c25d8065cb58eada7311954c4030320dfe7ab9e4eae65260e4e0737821
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 621.0 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manyli

In [ ]:
from rouge_score import rouge_scorer
from IPython.display import display
from nltk.translate import meteor
from nltk.tokenize import word_tokenize
import bert_score
import nltk
import os
import warnings
import logging

warnings.filterwarnings('ignore')
logging.getLogger("transformers").setLevel(logging.ERROR)

nltk.download('wordnet')
nltk.download('punkt')


predList = os.listdir("/content/predizioni/")
with open("risultatiCompleti.txt","w") as rc:
  for predName in predList:
    count = 0
    with open("predizioni/"+predName, 'r') as file:
      llavaPred = file.readlines()



    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


    res = pd.DataFrame(columns=['Reference', 'Candidate', 'rouge1', 'rouge2', 'rougeL','meteor','bertScoreP','bertScoreR','bertScoreF1'])

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    meteor_scores = []
    bertScoreP = []
    bertScoreR = []
    bertScoreF1 = []

    for pred in llavaPred:
      found = False
      for index,row in df.iterrows():
        query = row["Query"]
        if query.lower() in pred.lower():
          found = True
          count+=1
          break
      if "Error:" in pred or found == False:
        continue
      predAns = pred.split(':')[-1].split('.')[0].strip()
      score = scorer.score(row["Answer"], predAns)
      meteorVal= meteor([word_tokenize(row["Answer"].lower())],word_tokenize(predAns.lower()))
      P, R, F1 = bert_score.score([predAns], [row["Answer"]], lang="en",model_type="roberta-large", verbose=False)
      rouge1_scores.append(score['rouge1'].fmeasure)
      rouge2_scores.append(score['rouge2'].fmeasure)
      rougeL_scores.append(score['rougeL'].fmeasure)
      bertScoreP.append(P.mean())
      bertScoreR.append(R.mean())
      bertScoreF1.append(F1.mean())
      meteor_scores.append(meteorVal)

      #res = res.append({'Reference': row["Answer"], 'Candidate': predAns, 'Score1': rouge1_scores, 'Score2': rouge2_scores, 'Score3': rougeL_scores}, ignore_index=True)
      res.loc[len(res)] = [row["Answer"], predAns, rouge1_scores, rouge2_scores, rougeL_scores,meteorVal,P.mean(),R.mean(),F1.mean()]

    avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
    avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
    avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)
    avg_meteor = sum(meteor_scores) / len(meteor_scores)
    avg_bertScoreP = sum(bertScoreP) / len(bertScoreP)
    avg_bertScoreR = sum(bertScoreR) / len(bertScoreR)
    avg_bertScoreF1 = sum(bertScoreF1) / len(bertScoreF1)

    print(f'Average ROUGE-1 F1 Score: {avg_rouge1:.4f}')
    print(f'Average ROUGE-2 F1 Score: {avg_rouge2:.4f}')
    print(f'Average ROUGE-L F1 Score: {avg_rougeL:.4f}')
    print(f'Average METEOR Score: {avg_meteor:.4f}')
    print(f'Average BERT P Score: {avg_bertScoreP:.4f}')
    print(f'Average BERT R Score: {avg_bertScoreR:.4f}')
    print(f'Average BERT F1 Score: {avg_bertScoreF1:.4f}')

    rc.write(f"{predName}"+'\n')
    rc.write(f'Average ROUGE-1 F1 Score: {avg_rouge1:.4f}'+"\n")
    rc.write(f'Average ROUGE-2 F1 Score: {avg_rouge2:.4f}'+"\n")
    rc.write(f'Average ROUGE-L F1 Score: {avg_rougeL:.4f}'+"\n")
    rc.write(f'Average METEOR Score: {avg_meteor:.4f}'+"\n")
    rc.write(f'Average BERT P Score: {avg_bertScoreP:.4f}'+"\n")
    rc.write(f'Average BERT R Score: {avg_bertScoreR:.4f}'+"\n")
    rc.write(f'Average BERT F1 Score: {avg_bertScoreF1:.4f}'+"\n")
    print(count)
    res.to_csv(f"{predName}"+'\n')
    display(res)



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Average ROUGE-1 F1 Score: 0.5898
Average ROUGE-2 F1 Score: 0.3661
Average ROUGE-L F1 Score: 0.5259
Average METEOR Score: 0.5295
Average BERT P Score: 0.9249
Average BERT R Score: 0.9328
Average BERT F1 Score: 0.9287
50


,Reference,Candidate,rouge1,rouge2,rougeL,meteor,bertScoreP,bertScoreR,bertScoreF1
0,You chopped green peppers.,You chopped celery and onions in the video,"[0.3333333333333333, 0.5882352941176471, 0.875...","[0.2, 0.26666666666666666, 0.7142857142857143,...","[0.3333333333333333, 0.47058823529411764, 0.87...",0.353774,tensor(0.8827),tensor(0.9203),tensor(0.9011)
1,You ate porridge and beans from a bowl.,You ate a bowl of cereal and a banana,"[0.3333333333333333, 0.5882352941176471, 0.875...","[0.2, 0.26666666666666666, 0.7142857142857143,...","[0.3333333333333333, 0.47058823529411764, 0.87...",0.413333,tensor(0.9346),tensor(0.9276),tensor(0.9311)
2,There were eight pies in the brown plate.,There were three pies in the brown plate,"[0.3333333333333333, 0.5882352941176471, 0.875...","[0.2, 0.26666666666666666, 0.7142857142857143,...","[0.3333333333333333, 0.47058823529411764, 0.87...",0.777345,tensor(0.9872),tensor(0.9887),tensor(0.9880)
3,You put your phone on the kitchen counter.,The person holding the cell phone is standing ...,"[0.3333333333333333, 0.5882352941176471, 0.875...","[0.2, 0.26666666666666666, 0.7142857142857143,...","[0.3333333333333333, 0.47058823529411764, 0.87...",0.163043,tensor(0.8743),tensor(0.8956),tensor(0.8848)
4,The first tool you used was a screwdriver.,The man used a green tool to operate the machi...,"[0.3333333333333333, 0.5882352941176471, 0.875...","[0.2, 0.26666666666666666, 0.7142857142857143,...","[0.3333333333333333, 0.47058823529411764, 0.87...",0.271739,tensor(0.8848),tensor(0.8966),tensor(0.8906)
5,You put oil in the dough.,"You put a cup of dough into the mixer, which w...","[0.3333333333333333, 0.5882352941176471, 0.875...","[0.2, 0.26666666666666666, 0.7142857142857143,...","[0.3333333333333333, 0.47058823529411764, 0.87...",0.462963,tensor(0.8846),tensor(0.9388),tensor(0.9109)
6,You talked to a brunette woman in an orange sw...,You talked to a woman at the ATM place,"[0.3333333333333333, 0.5882352941176471, 0.875...","[0.2, 0.26666666666666666, 0.7142857142857143,...","[0.3333333333333333, 0.47058823529411764, 0.87...",0.613426,tensor(0.9732),tensor(0.9162),tensor(0.9438)
7,The egg was in the fridge before you picked it.,The egg was in the fridge before you picked it,"[0.3333333333333333, 0.5882352941176471, 0.875...","[0.2, 0.26666666666666666, 0.7142857142857143,...","[0.3333333333333333, 0.47058823529411764, 0.87...",0.916972,tensor(0.9940),tensor(0.9945),tensor(0.9943)
8,The round brush was on the floor before the pe...,The round brush was before being picked up by ...,"[0.3333333333333333, 0.5882352941176471, 0.875...","[0.2, 0.26666666666666666, 0.7142857142857143,...","[0.3333333333333333, 0.47058823529411764, 0.87...",0.447226,tensor(0.9317),tensor(0.9256),tensor(0.9286)
9,The words written in the box close to the wall...,"The box near the wall had the words ""2 gays 1 ...","[0.3333333333333333, 0.5882352941176471, 0.875...","[0.2, 0.26666666666666666, 0.7142857142857143,...","[0.3333333333333333, 0.47058823529411764, 0.87...",0.396641,tensor(0.9241),tensor(0.8824),tensor(0.9028)
